<a href="https://colab.research.google.com/github/Torikul385/NLP/blob/main/Eng_to_Bangla_Translation_%5Bkeras%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install -q --upgrade tensorflow
!!pip install -q --upgrade keras

['\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/1.1 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━\x1b \x1b0.6/1.1 MB\x1b \x1b22.9 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b1.1/1.1 MB\x1b \x1b26.1 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.1/1.1 MB\x1b \x1b13.3 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h']

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import string
import re

import keras
from keras import layers, ops

import tensorflow as tf

In [3]:
!!wget -q http://www.manythings.org/anki/ben-eng.zip

[]

In [4]:
!!unzip -q ben-eng.zip

[]

In [5]:
!!head -20 ben.txt

['Go.\tযাও।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545004 (tanay)',
 'Go.\tযান।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545005 (tanay)',
 'Go.\tযা।\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545006 (tanay)',
 'Run!\tপালাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548781 (tanay)',
 'Run!\tপালান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5548783 (tanay)',
 'Who?\tকে?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #5548787 (tanay)',
 'Wow!\tবাহ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #11026314 (robingaspi)',
 'Fire!\tআগুন!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #3232240 (tanay)',
 'Help!\tবাঁচাও!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548780 (tanay)',
 'Help!\tবাঁচান!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #5548782 (tanay)',
 'Jump.\tঝাঁপ দাও।\tCC-BY 2.

###Process data

In [6]:
pairs = []

with open("/content/ben.txt") as f:
  lines = f.readlines()

for line in lines:
   line = line.replace('\n', "").split("\t")
   eng = line[0]
   ben = "[start] " + line[1] + " [end]"

   pairs.append([eng, ben])


In [7]:
pairs = np.array(pairs)

indx = np.random.permutation(len(pairs))
pairs = pairs[indx]

for pair in pairs[:10]:
  print(pair[0], pair[1])

We're at war. [start] আমরা যুদ্ধে আছি। [end]
He looks pale. [start] ওকে ফ্যাকাসে দেখাচ্ছে। [end]
I don't want to go there again. [start] আমি ওখানে আবার যেতে চাই না। [end]
Oil is scarce in this country. [start] এই দেশে তেলের অভাব। [end]
Fire! [start] আগুন! [end]
He attends the same school that I do. [start] আমি যে স্কুলে যা, সে ঐ স্কুলেই যায়। [end]
I'm not sure that I've got it right. [start] আমি নিশ্চিত নই যে আমি এটা ঠিক করেছি। [end]
Birds fly. [start] পাখি ওড়ে। [end]
Don't move! [start] সরেন না! [end]
Babies don't stay babies forever. [start] শিশুরা চিরকাল শিশু থাকে না। [end]


In [8]:
train_size = int(0.9 * len(pairs))
val_size = int(0.05 * len(pairs))

train_pairs = pairs[:train_size]
test_pairs = pairs[train_size: train_size + val_size]
val_pairs = pairs[train_size + val_size:]

print(f"Total pairs : {len(pairs)}")
print(f"Train Pairs : {len(train_pairs)}")
print(f"Test Pairs : {len(test_pairs)}")
print(f"Validation Pairs : {len(val_pairs)}")

Total pairs : 6509
Train Pairs : 5858
Test Pairs : 325
Validation Pairs : 326


###Vectorization

In [9]:
strip_chars = string.punctuation + "।"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(text):
  text = tf.strings.regex_replace(text, f"[{re.escape(strip_chars)}]", "")
  return text

vocab_size = 15000
seq_len = 20


eng_vect = layers.TextVectorization(
    max_tokens = vocab_size,
    output_sequence_length = seq_len
)

ben_vect = layers.TextVectorization(
    max_tokens = vocab_size,
    standardize=custom_standardization,
    output_sequence_length = seq_len+1
)

eng_texts = [pair[0] for pair in train_pairs]
ben_texts = [pair[1] for pair in train_pairs]

eng_vect.adapt(eng_texts)
ben_vect.adapt(ben_texts)

###Make dataset

In [10]:
def preprocess_text(eng, ben):
  eng = eng_vect(eng)
  ben = ben_vect(ben)

  return (
      {
          "encoder_inputs": eng,
          "decoder_inputs": ben[:,:-1]
      },
      ben[:,1:]
  )

batch_size = 64
def make_dataset(pairs):
  eng_texts, ben_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  ben_texts = list(ben_texts)

  ds = tf.data.Dataset.from_tensor_slices((eng_texts, ben_texts))
  ds = ds.batch(batch_size).map(preprocess_text)
  ds = ds.cache().shuffle(2048).prefetch(16)

  return ds

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [11]:
for inputs, targets in train_ds.take(1):
  print(inputs['encoder_inputs'].shape)
  print(inputs['decoder_inputs'].shape)
  print(targets.shape)

(64, 20)
(64, 20)
(64, 20)


In [20]:
class Encoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads

    self.attention = layers.MultiHeadAttention(
        num_heads = num_heads,
        key_dim = embed_dim
    )
    self.dense_proj = keras.Sequential(
        [
            layers.Dense(dense_dim, activation='relu'),
            layers.Dense(embed_dim)
        ]
    )

    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

    self.support_masking = True

  def call(self, inputs, mask=None):
    if mask is not None:
      padding_mask = ops.cast(mask[:,None,:], dtype='int32')
    else:
      padding_mask = None

    attention_output = self.attention(
        query=inputs,
        key=inputs,
        value=inputs,
        attention_mask=padding_mask
    )

    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return  self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        'embed_dim' : self.embed_dim,
        'vocab_size' : self.vocab_size,
        'num_heads': self.num_heads
    })
    return config


In [19]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, seq_len, vocab_size, embed_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embed = layers.Embedding(
        input_dim = vocab_size,
        output_dim = embed_dim
    )
    self.pos_embed = layers.Embedding(
        input_dim = seq_len,
        output_dim = embed_dim
    )

    self.embed_dim = embed_dim
    self.vocab_size = vocab_size
    self.seq_len = seq_len

  def call(self, inputs):
    length = ops.shape(inputs)[-1]
    positions = ops.arange(0, length, 1)

    embed_token = self.token_embed(inputs)
    embed_pos = self.pos_embed(positions)
    return embed_token + embed_pos

  def compute_mask(self, inputs, mask=None):
    if mask is None:
      return None
    else:
      return ops.not_equal(inputs, 0)


  def get_config(self):
    config = super.get_config()
    config.update({
        'seq_len': self.seq_len,
        'vocab_size': self.vocab_size,
        'embed_dim': self.embed_dim
    })
    return config

In [21]:
class Decoder(layers.Layer):
  def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.latent_dim = latent_dim
    self.num_heads = num_heads

    self.attention_1 = layers.MultiHeadAttention(
        num_heads = num_heads,
        key_dim = embed_dim
    )
    self.attention_2 = layers.MultiHeadAttention(
        num_heads = num_heads,
        key_dim = embed_dim
    )

    self.dense_proj = keras.Sequential([
        layers.Dense(latent_dim, activation='relu'),
        layers.Dense(embed_dim)
    ])

    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
    self.layernorm_3 = layers.LayerNormalization()

    self.supports_mask = True

  def call(self, inputs, encoder_outputs, mask=None):
    causal_mask = self.get_causal_mask(inputs)

    if mask is not None:
      padding_mask = ops.cast(mask[:,None,:], dtype='int32')
      padding_mask = ops.minimum(padding_mask, causal_mask)

    else:
      padding_mask = None

    attention_output_1 = self.attention_1(
        query=inputs,
        key=inputs,
        value=inputs,
        attention_mask = causal_mask
    )
    out_1 = self.layernorm_1(inputs+attention_output_1)
    attention_2 = self.attention_2(
        query=inputs,
        key=inputs,
        value=inputs,
        attention_mask = padding_mask
    )

    out_2 = self.layernorm_2(out_1 + attention_2)
    proj_out = self.dense_proj(out_2)
    return self.layernorm_3(proj_out + out_2)

  def get_causal_mask(self, inputs):
    input_shape = ops.shape(inputs)
    batch_size, seq_len = input_shape[0], input_shape[1]

    i = ops.arange(seq_len)[:,None]
    j = ops.arange(seq_len)
    mask = ops.cast(i>=j, dtype='int32')
    mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
    mult = ops.concatenate([ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1,1])], axis=0)

    return ops.tile(mask, mult)

  def get_config(self):
    config = super().get_config()
    config.update({
        'embed_dim': self.embed_dim,
        'num_heads': self.num_heads,
        'latent_dim': self.latent_dim
    })
    return config

In [22]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype='int64', name='encoder_inputs')
x = PositionalEmbedding(seq_len, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = Encoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype='int64', name='decoder_inputs')
encoder_seq_inputs = keras.Input(shape=(None,embed_dim), name='decoder_state_inputs')
x = PositionalEmbedding(seq_len, vocab_size, embed_dim)(decoder_inputs)
x = Decoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation='softmax')(x)
decoder = keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name='transformer')

In [16]:
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding      │ (None, None, 256)      │      3,845,120 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder (Encoder)         │ (None, None, 256)      │      3,155,456 │ positional_embedding[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, None, 15000)    │     12,959,640 │ decoder_inputs[0][0],  │
│                           │                        │                │ encoder[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
transformer.compile(
    optimizer = keras.optimizers.RMSprop(),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [24]:
transformer.fit(train_ds, epochs=15, validation_data = val_ds)

Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 23s 154ms/step - accuracy: 0.6961 - loss: 3.8393 - val_accuracy: 0.7557 - val_loss: 1.7261
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.7579 - loss: 1.6766 - val_accuracy: 0.7583 - val_loss: 1.6563
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.7648 - loss: 1.5947 - val_accuracy: 0.7535 - val_loss: 1.6714
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.7706 - loss: 1.5094 - val_accuracy: 0.7571 - val_loss: 1.5644
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.7779 - loss: 1.4256 - val_accuracy: 0.7758 - val_loss: 1.4817
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.7829 - loss: 1.3421 - val_accuracy: 0.7704 - val_loss: 1.4608
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.7912 - loss: 1.2691 - val_accuracy: 0.7894 - val_loss: 1.3922
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.7991 - loss: 1.1758 - val_accuracy: 0.7842 -

In [ ]:
ban_vocab = ben_vect.get_vocabulary()
ban_index_lookup = dict(zip(range(len(ban_vocab)), ban_vocab))
max_decoded_sentence_length = 20

def decode_sentence(input_sentence):
  tokenized_input_sentence = eng_vect([input_sentence])

  decoded_sentence = "[start]"

  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = ben_vect([decoded_sentence])[:,:-1]
    preds = transformer([tokenized_input_sentence, tokenized_target_sentence])

    sampled_token_index = ops.convert_to_numpy(
        ops.argmax(preds[0,i,:])
    ).item(0)
    sampled_token = ban_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token

    if sampled_token == "[end]":
      break

  return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for i in range(10):
  input_sentence = test_eng_texts[i]
  translated = decode_sentence(input_sentence)

  print(f"\nEnglish : {input_sentence}")
  print(f"Bangla : {translated}")